In [19]:
import pulp
import pandas as pd
import io
import itertools

In [5]:
# 輸送問題

In [44]:
supply = {1:100, 2:180}
demand = {1:120, 2:40, 3:80}

COSTS = io.StringIO("""
S,D,C
1,1,10
1,2,6
1,3,16
2,1,8
2,2,8
2,3,10
""")
df_costs = pd.read_csv(COSTS)

In [45]:
df_costs['Var'] = [pulp.LpVariable(f'x{i}{j}', 0) for i, j in itertools.product(supply.keys(), demand.keys())]

In [46]:
df_costs

,S,D,C,Var
0,1,1,10,x11
1,1,2,6,x12
2,1,3,16,x13
3,2,1,8,x21
4,2,2,8,x22
5,2,3,10,x23


In [50]:
pl = pulp.LpProblem('problem_logistics')
pl += pulp.lpDot(df_costs.C, df_costs.Var), 'objective_function'
for i, v in df_costs.groupby('S'):
    pl += pulp.lpSum(v.Var) <= supply[i]
for j, v in df_costs.groupby('D'):
    pl += pulp.lpSum(v.Var) == demand[j]
pl

problem_logistics:
MINIMIZE
10*x11 + 6*x12 + 16*x13 + 8*x21 + 8*x22 + 10*x23 + 0
SUBJECT TO
_C1: x11 + x12 + x13 <= 100

_C2: x21 + x22 + x23 <= 180

_C3: x11 + x21 = 120

_C4: x12 + x22 = 40

_C5: x13 + x23 = 80

VARIABLES
x11 Continuous
x12 Continuous
x13 Continuous
x21 Continuous
x22 Continuous
x23 Continuous

In [51]:
result = pl.solve()
pulp.LpStatus[result]

'Optimal'

In [52]:
pulp.value(pl.objective)

2040.0

In [53]:
for v in pl.variables():
    print(f"{v} = {pulp.value(v)}")

x11 = 20.0
x12 = 40.0
x13 = 0.0
x21 = 100.0
x22 = 0.0
x23 = 80.0


In [54]:
# 割り当て問題

In [55]:
CSV = io.StringIO("""
Actor,Role,Score
1,1,3
1,2,6
1,3,9
1,4,8
2,1,6
2,2,3
2,3,2
2,4,4
3,1,9
3,2,3
3,3,2
3,4,5
4,1,6
4,2,2
4,3,3
4,4,8
""")
df_roles = pd.read_csv(CSV)

In [62]:
df_roles['Var'] = [pulp.LpVariable(f'x{i + 1}{j + 1}', cat = pulp.LpBinary) for i, j in itertools.product(range(4), range(4))]

In [72]:
pa = pulp.LpProblem('problem_assign', sense = pulp.LpMaximize)
# Varには0-1で値が入るため、スコアとVarの内積＝スコアの総和になる
pa += pulp.lpDot(df_roles.Score, df_roles.Var), 'objective_fucntion'
for _, v in df_roles.groupby('Actor'):
    pa += pulp.lpSum(v.Var) == 1
for _, v in df_roles.groupby('Role'):
    pa += pulp.lpSum(v.Var) == 1
pa

problem_assign:
MAXIMIZE
3*x11 + 6*x12 + 9*x13 + 8*x14 + 6*x21 + 3*x22 + 2*x23 + 4*x24 + 9*x31 + 3*x32 + 2*x33 + 5*x34 + 6*x41 + 2*x42 + 3*x43 + 8*x44 + 0
SUBJECT TO
_C1: x11 + x12 + x13 + x14 = 1

_C2: x21 + x22 + x23 + x24 = 1

_C3: x31 + x32 + x33 + x34 = 1

_C4: x41 + x42 + x43 + x44 = 1

_C5: x11 + x21 + x31 + x41 = 1

_C6: x12 + x22 + x32 + x42 = 1

_C7: x13 + x23 + x33 + x43 = 1

_C8: x14 + x24 + x34 + x44 = 1

VARIABLES
0 <= x11 <= 1 Integer
0 <= x12 <= 1 Integer
0 <= x13 <= 1 Integer
0 <= x14 <= 1 Integer
0 <= x21 <= 1 Integer
0 <= x22 <= 1 Integer
0 <= x23 <= 1 Integer
0 <= x24 <= 1 Integer
0 <= x31 <= 1 Integer
0 <= x32 <= 1 Integer
0 <= x33 <= 1 Integer
0 <= x34 <= 1 Integer
0 <= x41 <= 1 Integer
0 <= x42 <= 1 Integer
0 <= x43 <= 1 Integer
0 <= x44 <= 1 Integer

In [73]:
result = pa.solve()
pulp.LpStatus[result]

'Optimal'

In [74]:
 pulp.value(pa.objective)

29

In [75]:
for v in pa.variables():
    print(f"{v} = {pulp.value(v)}")

x11 = 0
x12 = 0
x13 = 1
x14 = 0
x21 = 0
x22 = 1
x23 = 0
x24 = 0
x31 = 1
x32 = 0
x33 = 0
x34 = 0
x41 = 0
x42 = 0
x43 = 0
x44 = 1


In [70]:
df_roles['Val'] = df_roles.Var.apply(pulp.value)
df_roles[df_roles.Val > 0]

,Actor,Role,Score,Var,Val
0,1,1,3,x11,1
7,2,4,4,x24,1
10,3,3,2,x33,1
13,4,2,2,x42,1
